# 라이브러리 import

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
base_dir = "/gdrive/MyDrive/SSAFY-필드프로젝트/"

In [ ]:
!pip install pymysql

In [ ]:
!pip install fasttext

In [ ]:
!pip install konlpy
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [ ]:
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import urllib.request
import requests
import gensim
import fasttext
import fasttext.util
from tqdm import tqdm
from collections import namedtuple
from konlpy.tag import Mecab
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec, doc2vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

# DB에서 책 데이터 가져오기

In [ ]:
db = pymysql.connect(
    user='ssafy',
    passwd='ssafyssafy',
    host='field-ensemble.cqwzhdgaabxu.ap-northeast-2.rds.amazonaws.com',
    db='fieldproject',
    charset='utf8'
)

In [ ]:
books = []
with db:
    with db.cursor(pymysql.cursors.DictCursor) as cursor:
        # sql = '''
        #     SELECT book_isbn, book_title, book_author, book_publisher, book_contents, book_description FROM book LIMIT 40000;
        # '''
        sql = '''
            SELECT book_isbn, book_title, book_author, book_publisher, book_contents, book_description FROM book;
        '''

        cursor.execute(sql)
        books = cursor.fetchall()
books = pd.DataFrame(books)

In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_isbn         40000 non-null  int64 
 1   book_title        40000 non-null  object
 2   book_author       40000 non-null  object
 3   book_publisher    40000 non-null  object
 4   book_contents     40000 non-null  object
 5   book_description  40000 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.8+ MB


In [ ]:
books.head()

,book_isbn,book_title,book_author,book_publisher,book_contents,book_description,cleaned_contents,cleaned_description
0,2592576687009,에스콰이어 Esquire C형 2021.3 (표지 : 카이) (부록없음),에스콰이어 편집부 지음,허스트중앙(Hearst-Joongang),카이 14p,카이 14p,카이,카이
1,8100009954419,해동율경집,김혜승 엮음,선문출판사,제 1편 시송문\n\r\n천수경\r\n고왕경 \r\n몽수경\r\n관세음보살신주경\r...,제 1편 시송문\n\r\n천수경\r\n고왕경 \r\n몽수경\r\n관세음보살신주경\r...,제 편시 송문 천수경 고왕경 몽 수경 관세음보살 신주경 수생 경 반야심경 화엄경 약...,제 편시 송문 천수경 고왕경 몽 수경 관세음보살 신주경 수생 경 반야심경 화엄경 약...
2,8620000007001,"지장경 (우리말, 세로)",정일 외 편집부 지음,보련각,지장계청\r\n 1. 지장보살본원경\r\n 제1품 - 도리천궁에서 신통을 나투...,지장계청\r\n 1. 지장보살본원경\r\n 제1품 - 도리천궁에서 신통을 나투...,지장 계청 지장보살 본 원경 제품 도리천궁 에서 신통 나투 제품 분신 모이 제품 중...,지장 계청 지장보살 본 원경 제품 도리천궁 에서 신통 나투 제품 분신 모이 제품 중...
3,8801748053950,떴다 떴다 종이비행기 111,정규일 지음,종이나라,종이접기 기본기호와 약속 ... 3\n지은이의 말 ... 4\n「떴다! 떴다! 종이...,가장 잘 나는 종이비행기 기종 6종류와 디자인 색종이 111장으로 구성되어 있어 원...,종이접기 기본 기호 약속 지은이 말 떴 떴 종이비행기 대하 여 비행기 역사 항공 단...,가장 나 종이비행기 기종 종류 디자인 색종이 장 구성 되 어 있 어 원 디자인 원 ...
4,8801748063812,다면체 종이접기,종이나라 편집부 지음,종이나라,종이접기의 가치••••••••2\r\n목차•••••••••••••••••••••••...,다면체 종이접기를 하면 두뇌개발과 예술적 감각이 향상되고 자기효능감이 높아진다. 다...,종이접기 가치 목차 기본 기호 약속 삼각 육면체 정육면체 정육면체 정육면체 팔면체 ...,다면체 종이접기 면 두뇌 개발 예술 적 감각 향상 되 자기 효능감 높 아 진다 다면...


In [ ]:
mecab = Mecab() 
def preprocess(content):
    stopwords = {
        '도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게',
        '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'
    }

    content = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '', str(content))
    content = re.sub("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", '', str(content))
    content = re.sub("(http|ftp|https)://(?:[-\w.]|(?:\da-fA-F]{2}))+", '', str(content))
    content = re.sub("<[^>]*>", '', str(content))
    content = re.sub("[\r|\n]", '', str(content))
    content = re.sub("\(", '', str(content))
    content = re.sub("\)", '', str(content))

    content = mecab.morphs(content) # 토큰화
    content = [word for word in content if not word in stopwords] # 불용어 제거
    content = " ".join(content)
    return content

books['cleaned_title'] = books['book_title'].apply(preprocess)
books['cleaned_contents'] = books['book_contents'].apply(preprocess)
books['cleaned_description'] = books['book_description'].apply(preprocess)

In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88416 entries, 0 to 88415
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   book_isbn            88416 non-null  int64 
 1   book_title           88416 non-null  object
 2   book_author          88416 non-null  object
 3   book_publisher       88416 non-null  object
 4   book_contents        88416 non-null  object
 5   book_description     88416 non-null  object
 6   cleaned_title        88416 non-null  object
 7   cleaned_contents     88416 non-null  object
 8   cleaned_description  88416 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.1+ MB


In [ ]:
books.to_pickle(base_dir + "data/book_data.pkl")

# 워드 임베딩 (doc2vec)

In [ ]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm=1,            # PV-DBOW 알고리즘으로 학습
    dbow_words=1,    # skip-gram 방식으로 학습
    window=10,       # window size
    vector_size=100, # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=5,     # 5회 미만으로 나온 단어 무시
    min_alpha=0.025, # min learning-rate
    workers=4,       # 모델 학습시 사용할 thread 개수
    hs = 1,          # hierar chical softmax / default 0
    negative = 10    # negative sampling / default 5
)

In [ ]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument((c), [d]) for c, d in books[['cleand_title', 'book_title']].values]

In [ ]:
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dm/m,d100,n10,hs,w10,mc5,s0.001,t4)


In [ ]:
for epoch in tqdm(range(50)):
    # Doc2Vec 학습을 진행하는데 Learning rate를 계속 감소해주면서 학습을 진행해줍니다. 
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 50/50 [00:25<00:00,  1.95it/s]


In [ ]:
doc_vectorizer.docvecs.most_similar('14M 세계사 연표', topn=20)

In [ ]:
doc_vectorizer.docvecs.doctags

# 워드 임베딩 (Fasttext)

In [1]:
import fasttext
import fasttext.util

In [3]:
ko_model = fasttext.load_model("cc.ko.300.bin")

# 다 읽으려면 너무 시간이 오래 걸려서, 상위 10000개만 읽음.
# ko_model = KeyedVectors.load_word2vec_format(base_dir + 'models/cc.ko.300.vec', encoding="ISO-8859-1")
# ko_model = fasttext.load_model(base_dir + "models/cc.ko.300.vec")


In [4]:
# fasttext.util.reduce_model(ko_model, 100)
# ko_model.save_model('cc.en.100.bin')

In [ ]:
# load 된 model 자체가 <class 'gensim.models.keyedvectors.Word2VecKeyedVectors'
# print(f"Type of model: {type(ko_model)}")
# print(ko_model.most_similar('teach'))
# print(ko_model.similarity('teacher', 'teaches'))
# ko_model.wv.vocab
ko_model.get_nearest_neighbors('ㅅㅂ')

[(0.8350241184234619, 'ㅈㄴ'),
 (0.8186970949172974, 'ㅆㅂ'),
 (0.807837188243866, '좆나'),
 (0.7996234893798828, '아놔'),
 (0.7992235422134399, '존나'),
 (0.7984483242034912, 'ㅁㅊ'),
 (0.7939881682395935, '시팔'),
 (0.7901992797851562, '걔도'),
 (0.7843388319015503, '머야'),
 (0.7828365564346313, '죤나')]

In [ ]:
def fasttext_vectorize(book_list):
    book_embedding_list = []

    for book in book_list:
        book_embedding_list.append(ko_model.get_sentence_vector(book))
    return book_embedding_list

In [ ]:
book_embedding_list = fasttext_vectorize(books[['cleaned_title', 'cleaned_description']].apply(' '.join, axis=1))

print(len(book_embedding_list))

88416


In [ ]:
np.save(base_dir + 'data/embedding_matrix.npy', book_embedding_list)

In [ ]:
len(book_embedding_list[0])

100

In [ ]:
book_embedding_list = cosine_similarity(book_embedding_list, book_embedding_list)
print('코사인 유사도 매트릭스의 크기 :',book_embedding_list.shape)

코사인 유사도 매트릭스의 크기 : (39124, 39124)


In [ ]:
def recommendations(isbn):
    recommended_books = books[['book_isbn','book_title', 'book_author']]

    # 책의 ISBN을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(books.index, index = books['book_isbn']).drop_duplicates()    
    idx = indices[isbn]

    # 해당 책과 다른 책들의 cosine similarity 계산
    cos_similarity = cosine_similarity([book_embedding_list[idx]], book_embedding_list)

    # 입력된 책과 제목이 유사한 책 5개 선정.
    sim_scores = list(enumerate(cos_similarity[0]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    print(sim_scores)

    # 가장 유사한 책 5권의 인덱스
    book_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = recommended_books.iloc[book_indices].reset_index(drop=True)

    return recommend

In [ ]:
recommend = recommendations(9788930106924)
recommend

[(8026, 0.98234844), (26450, 0.9823119), (42719, 0.9821646), (12555, 0.98156804), (18699, 0.9807234)]


,book_isbn,book_title,book_author
0,9788932022505,가나,정용준 지음
1,9788961706889,뼈들이 노래한다,"숀 탠 지음, 황윤영 옮김"
2,9788986270433,즉흥연기,"키스 존스톤 지음, 이민아 옮김"
3,9788937484025,우스운 사랑들,"밀란 쿤데라 지음, 방미경 옮김"
4,9788954656696,무엇이든 가능하다,"엘리자베스 스트라우트 지음, 정연희 옮김"


# word2vec

In [ ]:
# urllib.request.urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", filename="GoogleNews-vectors-negative300.bin.gz")

# word2vec_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
# word2vec_model.build_vocab(corpus)
# word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True)
# word2vec_model.train(corpus, total_examples = word2vec_model.corpus_count, epochs = 15)

model = gensim.models.Word2Vec.load(base_dir + "models/ko.bin")

In [ ]:
result = model.wv.most_similar("강아지")
print(result)

[('고양이', 0.7290452718734741), ('거위', 0.7185635566711426), ('토끼', 0.7056223154067993), ('멧돼지', 0.6950401067733765), ('엄마', 0.6934334635734558), ('난쟁이', 0.6806551218032837), ('한마리', 0.6770296096801758), ('아가씨', 0.6750352382659912), ('아빠', 0.6729634404182434), ('목걸이', 0.6512460708618164)]


# 벡터화

In [ ]:
def vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = model[word]
                else:
                    doc2vec = doc2vec + model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [ ]:
document_embedding_list = vectors(books['cleaned_description'])
print('문서 벡터의 수 :',len(document_embedding_list))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


문서 벡터의 수 : 9851


# 코사인 유사도 계산

In [ ]:
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

코사인 유사도 매트릭스의 크기 : (9851, 9851)


# 책 소개가 가장 유사한 책 5개 찾기

In [ ]:
def recommendations(isbn):
    recommended_books = books[['book_title', 'book_author']]

    # 책의 제목을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(books.index, index = books['book_isbn']).drop_duplicates()    
    idx = indices[isbn]

    # 입력된 책과 줄거리(document embedding)가 유사한 책 5개 선정.
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]

    # 가장 유사한 책 5권의 인덱스
    book_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = recommended_books.iloc[book_indices].reset_index(drop=True)

    return recommend

In [ ]:
recommend = recommendations(9788920020346)
recommend

,book_title,book_author
0,원예학 (워크북 포함),문원.전창후.이지원 지음
1,가족자원관리학 (워크북 포함),김외숙.이기영 지음
2,글과 생각 (워크북 포함),박종성 외 지음
3,작업환경측정 (워크북 포함),박동욱 외 지음
4,글쓰기의 지도,한수영 지음
